In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-english/_about.txt
/kaggle/input/french-english/fra.txt


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , LSTM , Dense
import numpy as np



batch_size = 64

epochs = 100

latent_dim =256 #size of feature vector

num_samples = 10000

data_path = '/kaggle/input/french-english/fra.txt'

In [3]:
input_texts = []
target_texts = []

input_characters = set()
target_characters = set()

with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text , target_text , _ = line.split('\t')
    
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
input_texts[:5],target_texts[:5]

(['Go.', 'Hi.', 'Hi.', 'Run!', 'Run!'],
 ['\tVa !\n',
  '\tSalut !\n',
  '\tSalut.\n',
  '\tCours\u202f!\n',
  '\tCourez\u202f!\n'])

In [9]:
input_characters = sorted(list(input_characters))

target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)

num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])

max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print("number of samples" ,  len(input_texts))
print("max input seq length ",max_encoder_seq_length)
print('max decoder seq length',max_decoder_seq_length)

number of samples 10000
max input seq length  15
max decoder seq length 59


In [11]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])

target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [15]:
input_token_index , target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  '%': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '0': 9,
  '1': 10,
  '2': 11,
  '3': 12,
  '5': 13,
  '6': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '2': 15,
  '3': 16,
  '5': 17,
  '8': 18,
  '9': 19,
  ':': 20,
  '?'

In [16]:
#onehot encoder
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')

decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')


decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')


In [17]:
#one hot implementation - from above (continuation)

for i ,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1. 
    encoder_input_data[i,t+1:,input_token_index[' ']] =1.
    
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.
        
        if t>0:
            
            decoder_target_data[i,t-1,target_token_index[char]] = 1.
    
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i,t:,target_token_index[' ']] = 1.

In [18]:
encoder_input_data[0].shape

(15, 71)

In [22]:
#building model

#ENCODER
encoder_inputs = Input(shape=(None,num_encoder_tokens))

encoder = LSTM(latent_dim,return_state=True)

encoder_outputs , state_h , state_c = encoder(encoder_inputs)
# we dont need the encoder outputs one by one so we ignore it and only consider the hidden states

encoder_states = [state_h,state_c]



#DECODER
decoder_inputs = Input(shape=(None,num_decoder_tokens))

decoder_lstm = LSTM(latent_dim,return_sequences = True, return_state=True)

decoder_outputs , __ , _ = decoder_lstm(decoder_inputs,
                                 initial_state=encoder_states)

#FINAL LAYERS

decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

model.compile(optimizer='rmsprop' , loss = 'categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, None, 94)]   0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 256), (None, 335872      input_6[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 256),  359424      input_7[0][0]                    
                                                                 lstm_5[0][1]          

In [24]:
model.fit([encoder_input_data,decoder_input_data],
         decoder_target_data,
         batch_size=batch_size,
         epochs=120,
         validation_split=0.2,
         )

model.save('s2s')

Epoch 1/120
125/125 [==============================] - 2s 20ms/step - loss: 1.1784 - accuracy: 0.7214 - val_loss: 1.1081 - val_accuracy: 0.7113
Epoch 2/120
125/125 [==============================] - 2s 14ms/step - loss: 0.8743 - accuracy: 0.7673 - val_loss: 0.8520 - val_accuracy: 0.7610
Epoch 3/120
125/125 [==============================] - 2s 13ms/step - loss: 0.7051 - accuracy: 0.8033 - val_loss: 0.7308 - val_accuracy: 0.7897
Epoch 4/120
125/125 [==============================] - 2s 14ms/step - loss: 0.6123 - accuracy: 0.8223 - val_loss: 0.6651 - val_accuracy: 0.8038
Epoch 5/120
125/125 [==============================] - 2s 13ms/step - loss: 0.5564 - accuracy: 0.8375 - val_loss: 0.6287 - val_accuracy: 0.8134
Epoch 6/120
125/125 [==============================] - 2s 15ms/step - loss: 0.5161 - accuracy: 0.8487 - val_loss: 0.6007 - val_accuracy: 0.8207
Epoch 7/120
125/125 [==============================] - 2s 13ms/step - loss: 0.4851 - accuracy: 0.8568 - val_loss: 0.5761 - val_accuracy:

125/125 [==============================] - 2s 13ms/step - loss: 0.1022 - accuracy: 0.9678 - val_loss: 0.5806 - val_accuracy: 0.8711
Epoch 58/120
125/125 [==============================] - 2s 14ms/step - loss: 0.1002 - accuracy: 0.9685 - val_loss: 0.5885 - val_accuracy: 0.8712
Epoch 59/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0981 - accuracy: 0.9688 - val_loss: 0.5936 - val_accuracy: 0.8713
Epoch 60/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0959 - accuracy: 0.9696 - val_loss: 0.5963 - val_accuracy: 0.8705
Epoch 61/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0942 - accuracy: 0.9700 - val_loss: 0.6009 - val_accuracy: 0.8705
Epoch 62/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0923 - accuracy: 0.9705 - val_loss: 0.6038 - val_accuracy: 0.8706
Epoch 63/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0903 - accuracy: 0.9711 - val_loss: 0.6148 - val_accuracy: 0.870

125/125 [==============================] - 2s 13ms/step - loss: 0.0420 - accuracy: 0.9850 - val_loss: 0.7813 - val_accuracy: 0.8694
Epoch 114/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0409 - accuracy: 0.9854 - val_loss: 0.7874 - val_accuracy: 0.8696
Epoch 115/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0408 - accuracy: 0.9853 - val_loss: 0.7875 - val_accuracy: 0.8688
Epoch 116/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0403 - accuracy: 0.9853 - val_loss: 0.7909 - val_accuracy: 0.8682
Epoch 117/120
125/125 [==============================] - 2s 14ms/step - loss: 0.0403 - accuracy: 0.9852 - val_loss: 0.7991 - val_accuracy: 0.8679
Epoch 118/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0396 - accuracy: 0.9857 - val_loss: 0.8040 - val_accuracy: 0.8678
Epoch 119/120
125/125 [==============================] - 2s 13ms/step - loss: 0.0390 - accuracy: 0.9858 - val_loss: 0.7991 - val_accuracy:

In [ ]:
model.save("lang_trans.h5")

In [27]:
import keras
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [28]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
I